# Motivation

Previously ran with 9000 iterations, now to see if double the iterations (18,000) - how does it impact instruction and joules count? Expect to be 2X previous results

Summary: Joules still track with perf and instructions also increase 2X. The instruction count is still off by ~3X of the perf stat metrics but its measuring something consistently at least

In [7]:
for iter in [9000, 18000]:
    for round in range(0,3):
        fs = f"../results/runALU.ITER{iter}.PARALLEL1.ROUND{round}.START"
        fe = f"../results/runALU.ITER{iter}.PARALLEL1.ROUND{round}.END"


        with open(fe) as f:
            lines = f.readlines()
        for line in lines:
            if "microbench" in line and "kepler_container_cpu_instructions_total" in line:
                inse = float(line.split(' ')[1])
            if "microbench" in line and "kepler_container_cache_miss_total" in line:
                ce = float(line.split(' ')[1])
            if "microbench" in line and "kepler_container_cpu_cycles_total" in line:
                cyce = float(line.split(' ')[1])
            if "kepler_node_package_joules_total" in line and "dynamic" in line:
                dynpkge = float(line.split(' ')[1])
            if "kepler_node_package_joules_total" in line and "idle" in line:
                idlepkge = float(line.split(' ')[1])

        with open(fs) as f:
            lines = f.readlines()
        for line in lines:
            if "microbench" in line and "kepler_container_cpu_instructions_total" in line:
                inss = float(line.split(' ')[1])
            if "microbench" in line and "kepler_container_cache_miss_total" in line:
                cs = float(line.split(' ')[1])
            if "microbench" in line and "kepler_container_cpu_cycles_total" in line:
                cycs = float(line.split(' ')[1])
            if "kepler_node_package_joules_total" in line and "dynamic" in line:
                dynpkgs = float(line.split(' ')[1])
            if "kepler_node_package_joules_total" in line and "idle" in line:
                idlepkgs = float(line.split(' ')[1])


        print(f"ITER {iter} Run {round}:")
        print(f"\t kepler_container_cpu_instructions_total: {inse-inss}")
        print(f"\t kepler_container_cache_miss_total: {ce-cs}")
        print(f"\t kepler_container_cpu_cycles_total: {cyce-cycs}")
        print(f"\t kepler_node_package_joules_total: {(idlepkge-idlepkgs)+(dynpkge-dynpkgs)}")
        print(f"\t\t dynamic: {dynpkge-dynpkgs}")
        print(f"\t\t idle: {idlepkge-idlepkgs}")
    
    

ITER 9000 Run 0:
	 kepler_container_cpu_instructions_total: 35352091069.0
	 kepler_container_cache_miss_total: 169268989.0
	 kepler_container_cpu_cycles_total: 57082676060.0
	 kepler_node_package_joules_total: 2242.6380000000026
		 dynamic: 996.9660000000003
		 idle: 1245.6720000000023
ITER 9000 Run 1:
	 kepler_container_cpu_instructions_total: 37237375419.0
	 kepler_container_cache_miss_total: 172931498.0
	 kepler_container_cpu_cycles_total: 61379455354.0
	 kepler_node_package_joules_total: 2183.742
		 dynamic: 984.2060000000001
		 idle: 1199.536
ITER 9000 Run 2:
	 kepler_container_cpu_instructions_total: 34842177717.0
	 kepler_container_cache_miss_total: 171881549.0
	 kepler_container_cpu_cycles_total: 58801940833.0
	 kepler_node_package_joules_total: 2230.8910000000024
		 dynamic: 985.219
		 idle: 1245.6720000000023
ITER 18000 Run 0:
	 kepler_container_cpu_instructions_total: 71919723696.0
	 kepler_container_cache_miss_total: 344573712.0
	 kepler_container_cpu_cycles_total: 11348044

## Perf
```
hand32@node1:~/peakler/experiments/microbenchmarks$ for ((i=0;i<3;i++)); do perf stat -a -e instructions,cache-misses,cpu-cycles,power/energy-pkg/,power/energy-ram/ taskset -c 7 ./build/runALU 9000; done

 Performance counter stats for 'system wide':

   120,783,508,077        instructions              #    0.58  insn per cycle           (85.71%)
       335,151,980        cache-misses                                                  (85.72%)
   206,882,054,625        cpu-cycles                                                    (85.72%)
          2,206.95 Joules power/energy-pkg/
            484.82 Joules power/energy-ram/

      77.582574409 seconds time elapsed


 Performance counter stats for 'system wide':

   119,879,477,744        instructions              #    0.66  insn per cycle           (85.72%)
       329,221,432        cache-misses                                                  (85.72%)
   182,137,004,374        cpu-cycles                                                    (83.93%)
          2,206.53 Joules power/energy-pkg/
            483.16 Joules power/energy-ram/

      77.353079508 seconds time elapsed


 Performance counter stats for 'system wide':

   120,735,405,751        instructions              #    0.52  insn per cycle           (85.72%)
       334,831,649        cache-misses                                                  (85.72%)
   233,539,606,849        cpu-cycles                                                    (73.21%)
          2,214.79 Joules power/energy-pkg/
            483.48 Joules power/energy-ram/

      77.377628008 seconds time elapsed

hand32@node1:~/peakler/experiments/microbenchmarks$ for ((i=0;i<3;i++)); do perf stat -a -e instructions,cache-misses,cpu-cycles,power/energy-pkg/,power/energy-ram/ taskset -c 7 ./build/runALU 18000; done

 Performance counter stats for 'system wide':

   240,309,168,548        instructions              #    0.58  insn per cycle           (85.72%)
       666,778,183        cache-misses                                                  (85.72%)
   411,996,333,234        cpu-cycles                                                    (85.71%)
          4,425.40 Joules power/energy-pkg/
            968.06 Joules power/energy-ram/

     154.962687714 seconds time elapsed


 Performance counter stats for 'system wide':

   239,519,872,636        instructions              #    0.58  insn per cycle           (85.72%)
       664,424,787        cache-misses                                                  (85.72%)
   411,654,931,919        cpu-cycles                                                    (85.71%)
          4,426.89 Joules power/energy-pkg/
            966.01 Joules power/energy-ram/

     154.717036069 seconds time elapsed


 Performance counter stats for 'system wide':

   241,598,920,784        instructions              #    0.58  insn per cycle           (85.71%)
       669,690,200        cache-misses                                                  (85.71%)
   413,419,490,326        cpu-cycles                                                    (85.72%)
          4,432.70 Joules power/energy-pkg/
            966.94 Joules power/energy-ram/

     154.725924187 seconds time elapsed
```